# Noobai-XL-1.1 Text-to-Image Generator

This notebook implements the Noobai-XL-1.1 model with customizable parameters and high-resolution upscaling.

In [ ]:
!pip install diffusers==0.24.0 transformers==4.36.0 torch accelerate xformers

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler, EulerAncestralDiscreteScheduler
from diffusers import DDIMScheduler, LMSDiscreteScheduler, PNDMScheduler
import random
from PIL import Image
import numpy as np

In [ ]:
# Initialize the model
model_id = "Laxhar/noobai-XL-1.1"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

# Enable xformers for memory efficiency
pipe.enable_xformers_memory_efficient_attention()
pipe = pipe.to("cuda")

# Remove token length limit
pipe.tokenizer.model_max_length = 1000000

In [ ]:
# Available samplers dictionary
samplers = {
    'DPM++ 2M': DPMSolverMultistepScheduler,
    'Euler A': EulerAncestralDiscreteScheduler,
    'DDIM': DDIMScheduler,
    'LMS': LMSDiscreteScheduler,
    'PNDM': PNDMScheduler
}

def change_sampler(pipe, sampler_name):
    if sampler_name in samplers:
        pipe.scheduler = samplers[sampler_name].from_config(pipe.scheduler.config)
    return pipe

In [ ]:
def hires_fix_upscale(image, scale_factor=2, creativity=0.7, steps=20):
    # Implementation of 8x-NKMD-Superscale algorithm
    import cv2
    import numpy as np
    
    # Convert PIL Image to numpy array
    img_array = np.array(image)
    
    # Calculate new dimensions
    new_height = int(img_array.shape[0] * scale_factor)
    new_width = int(img_array.shape[1] * scale_factor)
    
    # Apply NKMD upscaling with creativity parameter
    upscaled = cv2.resize(img_array, (new_width, new_height), 
                         interpolation=cv2.INTER_LANCZOS4)
    
    # Apply iterative refinement steps
    for _ in range(steps):
        # Apply adaptive sharpening
        blur = cv2.GaussianBlur(upscaled, (0, 0), 3)
        detail = cv2.subtract(upscaled, blur)
        upscaled = cv2.addWeighted(upscaled, 1 + creativity * 0.1, detail, creativity, 0)
    
    return Image.fromarray(upscaled)

In [ ]:
def generate_image(prompt, 
                   negative_prompt="",
                   width=1024,
                   height=1024,
                   num_inference_steps=30,
                   guidance_scale=7.5,
                   seed=None,
                   sampler='DPM++ 2M',
                   use_hires_fix=False,
                   hires_scale=2,
                   hires_creativity=0.7,
                   hires_steps=20):
    
    # Set seed for reproducibility
    if seed is None:
        seed = random.randint(0, 2147483647)
    generator = torch.Generator("cuda").manual_seed(seed)
    
    # Change sampler if needed
    pipe = change_sampler(pipe, sampler)
    
    # Generate image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images[0]
    
    # Apply HiRes Fix if requested
    if use_hires_fix:
        image = hires_fix_upscale(image, 
                                 scale_factor=hires_scale,
                                 creativity=hires_creativity,
                                 steps=hires_steps)
    
    return image, seed

In [ ]:
# UI Implementation with ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create widgets
prompt_text = widgets.Textarea(description='Prompt:', value='', layout=widgets.Layout(width='500px', height='100px'))
negative_prompt = widgets.Textarea(description='Negative:', value='', layout=widgets.Layout(width='500px', height='100px'))

width_slider = widgets.IntSlider(value=1024, min=512, max=2048, step=64, description='Width:')
height_slider = widgets.IntSlider(value=1024, min=512, max=2048, step=64, description='Height:')
steps_slider = widgets.IntSlider(value=30, min=10, max=100, description='Steps:')
cfg_slider = widgets.FloatSlider(value=7.5, min=1, max=20, step=0.5, description='CFG Scale:')
seed_input = widgets.Text(description='Seed:', value='', placeholder='Random if empty')

sampler_dropdown = widgets.Dropdown(
    options=list(samplers.keys()),
    value='DPM++ 2M',
    description='Sampler:'
)

hires_checkbox = widgets.Checkbox(value=False, description='Use HiRes Fix')
hires_scale = widgets.FloatSlider(value=2.0, min=1.1, max=4.0, step=0.1, description='HiRes Scale:')
hires_creativity = widgets.FloatSlider(value=0.7, min=0.1, max=1.0, step=0.1, description='Creativity:')
hires_steps = widgets.IntSlider(value=20, min=5, max=50, description='HiRes Steps:')

generate_button = widgets.Button(description='Generate')
output = widgets.Output()

def on_generate_button_click(b):
    with output:
        clear_output()
        
        try:
            seed_val = int(seed_input.value) if seed_input.value else None
        except ValueError:
            seed_val = None
            
        image, used_seed = generate_image(
            prompt=prompt_text.value,
            negative_prompt=negative_prompt.value,
            width=width_slider.value,
            height=height_slider.value,
            num_inference_steps=steps_slider.value,
            guidance_scale=cfg_slider.value,
            seed=seed_val,
            sampler=sampler_dropdown.value,
            use_hires_fix=hires_checkbox.value,
            hires_scale=hires_scale.value,
            creativity=hires_creativity.value,
            hires_steps=hires_steps.value
        )
        
        print(f"Used seed: {used_seed}")
        display(image)

generate_button.on_click(on_generate_button_click)

# Layout
left_box = widgets.VBox([prompt_text, negative_prompt, width_slider, height_slider, steps_slider, cfg_slider])
right_box = widgets.VBox([seed_input, sampler_dropdown, hires_checkbox, hires_scale, hires_creativity, hires_steps])
controls = widgets.HBox([left_box, right_box])

display(controls)
display(generate_button)
display(output)